In [91]:
# Import necessary libraries
import pandas as pd
import json
import string 

# Load the CSV dataset
df = pd.read_csv("semi_strut.csv")

# Display the first few rows of the dataset
df.head()

,Document ID,Content
0,1,"{\r\n ""title"": ""Introduction to Python"",\r\n..."
1,2,"{\r\n ""title"": ""Data Analysis with Pandas"",\..."
2,3,"{\r\n ""title"": ""Web Development with Flask"",..."
3,4,"{\r\n ""title"": ""Machine Learning with Scikit..."
4,5,"{\r\n ""title"": ""Data Visualization with Matp..."


In [108]:
# Tokenization function to extract terms from the JSON-like content
# Remember to exact both 
# 1 .Extract terms from various fields (title, author)
def tokenize_content(content):
    content_dict = json.loads(content)
    terms = []
    
    # Extract terms from various fields (title, author)
    terms.extend(content_dict.get("title", "").split())
    terms.extend(content_dict.get("author", "").split())
    terms.extend(content_dict.get("keyword", []))

    #extract terms from sections title and content
    sections = content_dict.get("sections",[])
    for section in sections:
        terms.extend(section.get("title","").split())
        terms.extend(section.get("content","").split())
        
    return terms
tokenize_content(df["Content"][0])
# 2. apply to all row in panda df , by create new column "Terms"
df["Terms"] = df["Content"].apply(tokenize_content)
df[["Document ID", "Terms"]]

# your code here.....

,Document ID,Terms
0,1,"[Introduction, to, Python, John, Doe, Getting,..."
1,2,"[Data, Analysis, with, Pandas, Jane, Smith, In..."
2,3,"[Web, Development, with, Flask, Mike, Johnson,..."
3,4,"[Machine, Learning, with, Scikit-Learn, Emily,..."
4,5,"[Data, Visualization, with, Matplotlib, Robert..."


In [109]:
# 4. Implement a preprocessing function that converts terms to lowercase, removes punctuation, and removes common stop words.
    # Create another new column "Terms_preprocessed"
def preprocess_terms(terms):
    # Define a set of common stop words
    stop_words = set([
        "a", "an", "the", "and", "is", "in", "it", "to", "of", "for", "on", "with", "as"
    ])
    
    # Remove punctuation and convert to lowercase
    terms = [term.lower().strip(string.punctuation) for term in terms]
    
    # Remove stop words
    terms = [term for term in terms if term not in stop_words]
    
    return terms

# your code here....
unique_terms = {term for doc in df["Terms"] for term in doc}
unique_terms


{'Analysis',
 'Basic',
 'Clark',
 'Creating',
 'Data',
 'DataFrames',
 'Davis',
 'Development',
 'Doe',
 'Emily',
 'Flask',
 'Getting',
 'Introduction',
 'Jane',
 'John',
 'Johnson',
 'Learning',
 'Machine',
 'Matplotlib',
 'Mike',
 'Pandas',
 'Pandas...',
 'Plots',
 'Python',
 'Python...',
 'Robert',
 'Routing',
 'Scikit-Learn',
 'Smith',
 'Started',
 'Supervised',
 'Syntax',
 'URL',
 'Visualization',
 'Web',
 'a',
 'analysis...',
 'and',
 'are',
 'artificial',
 'be',
 'can',
 'core',
 'created',
 'data',
 'defines',
 'easy',
 'for',
 'framework',
 'functions...',
 'in',
 'intelligence...',
 'is',
 'language...',
 'learning',
 'learning...',
 'library',
 'lightweight',
 'machine',
 'of',
 'patterns',
 'popular',
 'programming',
 'structure',
 'subfield',
 'syntax',
 'to',
 'type',
 'understand...',
 'using',
 'various',
 'versatile',
 'views...',
 'visualization...',
 'web',
 'with'}

In [110]:
# Initialize an empty inverted index dictionary
# Build the inverted index
# Initialize an empty inverted index dictionary
inverted_index = {}

# Build the inverted index
for index, row in df.iterrows():
    document_id = row["Document ID"]
    terms = row["Terms"]
    
    # Update the inverted index with terms and document IDs
    for term in terms:
        if term not in inverted_index:
            inverted_index[term] = set()
        inverted_index[term].add(document_id)

# Display the inverted index
inverted_index

{'Introduction': {1, 2, 4, 5},
 'to': {1, 2, 4, 5},
 'Python': {1},
 'John': {1},
 'Doe': {1},
 'Getting': {1, 3},
 'Started': {1, 3},
 'is': {1, 2, 3, 4, 5},
 'a': {1, 2, 3, 4, 5},
 'versatile': {1, 5},
 'programming': {1},
 'language...': {1},
 'Basic': {1},
 'Syntax': {1},
 'syntax': {1},
 'easy': {1},
 'understand...': {1},
 'Data': {2, 5},
 'Analysis': {2},
 'with': {2, 3, 4, 5},
 'Pandas': {2},
 'Jane': {2},
 'Smith': {2},
 'popular': {2},
 'library': {2, 5},
 'for': {2, 3, 5},
 'data': {2, 5},
 'analysis...': {2},
 'DataFrames': {2},
 'are': {2},
 'core': {2},
 'structure': {2},
 'in': {2, 3, 5},
 'Pandas...': {2},
 'Web': {3},
 'Development': {3},
 'Flask': {3},
 'Mike': {3},
 'Johnson': {3},
 'lightweight': {3},
 'web': {3},
 'framework': {3},
 'Python...': {3},
 'Routing': {3},
 'defines': {3},
 'URL': {3},
 'patterns': {3},
 'and': {3},
 'views...': {3},
 'Machine': {4},
 'Learning': {4},
 'Scikit-Learn': {4},
 'Emily': {4},
 'Davis': {4},
 'learning': {4},
 'subfield': {4},

In [111]:
# perform boolean operations on postings lists for Boolean search operations
posting_list = inverted_index['Visualization']
posting_list

{5}

In [112]:
# 1. "Python" OR "Pandas"
def or_postings(posting1, posting2):
    p1 = 0
    p2 = 0
    result = list()
    while p1 < len(posting1) and p2 < len(posting2):
        if posting1[p1] == posting2[p2]:
            result.append(posting1[p1])
            p1 += 1
            p2 += 1
        elif posting1[p1] > posting2[p2]:
            result.append(posting2[p2])
            p2 += 1
        else:
            result.append(posting1[p1])
            p1 += 1
    while p1 < len(posting1):
        result.append(posting1[p1])
        p1 += 1
    while p2 < len(posting2):
        result.append(posting2[p2])
        p2 += 1
    return result

In [113]:
pl_1 = list(inverted_index['Python'])
pl_2 = list(inverted_index['Pandas'])
or_postings(pl_1,pl_2)

[1, 2]

In [114]:
# 2. "Python" AND "data"
def and_postings(posting1, posting2):
    p1 = 0
    p2 = 0
    result = list()
    while p1 < len(posting1) and p2 < len(posting2):
        if posting1[p1] == posting2[p2]:
            result.append(posting1[p1])
            p1 += 1
            p2 += 1
        elif posting1[p1] == posting2[p2]:
            p2 += 1
        else:
            p1 += 1
    return result

In [115]:
pl_1 = list(inverted_index['Python'])
pl_2 = list(inverted_index['Data'])
and_postings(pl_1,pl_2)

[]